# 🎯 DQX Anomaly Detection Demo

This notebook demonstrates **ML-based anomaly detection** in DQX, from basic 101 usage to advanced features.

## 📚 What You'll Learn

1. **Basic 101**: Train a model and detect anomalies
2. **Understanding Scores**: Interpret anomaly scores and thresholds
3. **Feature Importance**: Which columns contribute to anomalies
4. **Drift Detection**: Monitor when models need retraining
5. **Advanced**: Ensemble models and temporal features

## 🔧 Prerequisites

- Databricks Runtime 13.3+ (for anomaly support)
- DQX installed with `anomaly` extras: `%pip install databricks-labs-dqx[anomaly]`


In [ ]:
# Setup and Imports
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
from datetime import datetime, timedelta
import numpy as np

from databricks.labs.dqx.anomaly import train, has_no_anomalies, AnomalyParams, IsolationForestConfig
from databricks.labs.dqx.engine import DQEngine
from databricks.labs.dqx.rule import DQDatasetRule
from databricks.sdk import WorkspaceClient

# Initialize
spark = SparkSession.builder.getOrCreate()
ws = WorkspaceClient()
dq_engine = DQEngine(ws)

# Set seed for reproducibility
np.random.seed(42)

print("✅ Setup complete!")
print(f"   Spark version: {spark.version}")


---

## Part 0: Auto-Discovery & Segments (Recommended) 🚀

**Zero-config anomaly detection** with automatic column and segment detection.

### Why Auto-Discovery?
- No manual column selection needed
- Automatically finds good segmentation (regions, categories, etc.)
- One-liner training for most use cases


In [ ]:
# Generate multi-region e-commerce data
# Each region has different typical patterns
data = []
for region in ["US", "EU", "APAC"]:
    base_amount = 100 if region == "US" else (150 if region == "EU" else 120)
    base_discount = 0.1 if region == "US" else (0.15 if region == "EU" else 0.12)
    
    for i in range(300):  # 300 records per region
        amount = base_amount + np.random.normal(0, 10)
        discount_rate = base_discount + np.random.normal(0, 0.02)
        quantity = np.random.randint(1, 10)
        
        data.append((
            region,
            f"product_{np.random.randint(1, 50)}",
            round(amount, 2),
            round(discount_rate, 3),
            quantity
        ))

df = spark.createDataFrame(data, "region string, product_id string, amount double, discount_rate double, quantity int")

# Save to table
df.write.mode("overwrite").saveAsTable("demo.orders_multiregion")

print(f"✅ Generated {df.count()} multi-region orders")
df.groupBy("region").count().orderBy("region").show()


In [ ]:
# Zero-config training: auto-discovers columns and segments!
model_uri = train(df=spark.table("demo.orders_multiregion"))

# DQX will print:
# "Auto-selected 3 columns: [amount, discount_rate, quantity]"
# "Auto-detected 1 segment column: [region] (3 total segments)"
# "Training segment 1/3: region=APAC"
# "Training segment 2/3: region=EU"
# "Training segment 3/3: region=US"

print("\n✅ Models trained successfully!")
print(f"Model URI: {model_uri}")


In [ ]:
# Score new data - auto-infers columns and segment_by from model!
test_data = [
    ("US", "product_10", 105.0, 0.11, 5),     # Normal US order
    ("US", "product_20", 500.0, 0.50, 100),   # Anomaly: unusual for US
    ("EU", "product_30", 155.0, 0.16, 4),     # Normal EU order
    ("EU", "product_40", 50.0, 0.01, 1),      # Anomaly: unusual for EU
    ("APAC", "product_50", 118.0, 0.13, 6),   # Normal APAC order
]

test_df = spark.createDataFrame(
    test_data,
    "region string, product_id string, amount double, discount_rate double, quantity int"
)

# Scoring automatically uses the correct regional model for each row!
checks = [
    has_no_anomalies()  # Zero config - infers everything from trained model!
]

result = dq_engine.apply_checks(test_df, checks)

# Show results
print("\n🎯 Anomaly Scores (by region-specific models):")
result.select("region", "amount", "discount_rate", "quantity", "anomaly_score").orderBy("region").show(truncate=False)

# Count anomalies
anomaly_count = result.filter(F.col("anomaly_score") > 0.5).count()
print(f"\n⚠️  Detected {anomaly_count} anomalies out of {result.count()} records")


### 🎉 Key Benefits of Auto-Discovery

1. **Zero Configuration**: No manual column selection needed
2. **Segment-Aware**: Automatically trains separate models per region/category
3. **Automatic Inference**: Scoring knows which segment model to use per row
4. **Scalable**: Works with any number of segments (auto-warns if >50)

**What happened behind the scenes:**
- DQX analyzed your data and selected numeric columns with variance
- Identified `region` as a good segment candidate (3 distinct values, >1000 rows per segment)
- Trained 3 separate models (US, EU, APAC) with regional baselines
- During scoring, each row was scored by its region-specific model

**For manual control**, you can still specify columns and segments explicitly (see Part 1 below).

---


---
## 📖 Part 1: Basic 101 - Your First Anomaly Detection

**Scenario**: You're monitoring API response times and request volumes. You want to automatically detect unusual patterns.

**The Problem**: Setting static thresholds (e.g., "response_time > 500ms") is brittle:
- Thresholds vary by time of day, day of week
- Multivariate anomalies are hard to catch (high latency + low volume might indicate an attack)

**The Solution**: ML-based anomaly detection learns "normal" behavior and flags deviations.


In [ ]:
# Generate sample API monitoring data (training set - normal behavior)
n_normal = 500

normal_response_times = np.clip(np.random.gamma(shape=2, scale=50, size=n_normal), 10, 300)
normal_request_volumes = np.clip(np.random.normal(100, 15, size=n_normal), 50, 150)

train_data = [
    (float(rt), float(rv)) 
    for rt, rv in zip(normal_response_times, normal_request_volumes)
]

train_df = spark.createDataFrame(train_data, schema="response_time_ms double, request_volume double")

print(f"✅ Generated {train_df.count()} training records")
print("\n📊 Sample of normal behavior:")
train_df.select(
    F.round("response_time_ms", 1).alias("response_time_ms"),
    F.round("request_volume", 1).alias("request_volume")
).show(5)


### Step 1: Train the Model

Training is simple - DQX handles all the complexity:


In [ ]:
# Train anomaly detection model
model_uri = train(
    df=train_df,
    columns=["response_time_ms", "request_volume"],
    model_name="api_monitoring_demo",
    registry_table="main.default.api_anomaly_models"
)

print(f"✅ Model trained successfully!")
print(f"   Model URI: {model_uri}")
print(f"   Registry: main.default.api_anomaly_models")


### Step 2: Score New Data

Now let's create some test data with both normal and anomalous patterns:


In [ ]:
# Generate test data with anomalies
test_data = [
    # Normal data points
    (100.0, 105.0, "normal"),
    (95.0, 98.0, "normal"),
    (110.0, 102.0, "normal"),
    # Anomalies: High latency + Normal volume
    (450.0, 100.0, "high_latency"),
    # Anomalies: Normal latency + Very low volume (potential DDoS)
    (100.0, 20.0, "low_volume_attack"),
    # Anomalies: Both high
    (480.0, 180.0, "system_overload"),
]

test_df = spark.createDataFrame(test_data, schema="response_time_ms double, request_volume double, label string")

print("📊 Test data (mixed normal + anomalies):")
test_df.show(truncate=False)


In [ ]:
# Apply anomaly detection check
checks = [
    DQDatasetRule(
        criticality="error",
        check_func=has_no_anomalies,
        check_func_kwargs={
            "columns": ["response_time_ms", "request_volume"],
            "model": "api_monitoring_demo",
            "registry_table": "main.default.api_anomaly_models",
            "score_threshold": 0.6,  # Adjust based on your tolerance
        }
    )
]

result_df = dq_engine.apply_checks(test_df, checks)

# Show results with anomaly scores
print("📊 Results with anomaly scores:")
result_df.select(
    F.round("response_time_ms", 1).alias("response_time"),
    F.round("request_volume", 1).alias("volume"),
    "label",
    F.round("anomaly_score", 3).alias("anomaly_score"),
    F.size("_errors").alias("error_count")
).orderBy(F.desc("anomaly_score")).show(truncate=False)

print("\n💡 Interpretation:")
print("   - Scores > 0.6: Flagged as anomalies")
print("   - Scores < 0.6: Considered normal")
print("   - Higher scores = more unusual")


---
## 📊 Part 2: Understanding Scores and Thresholds

**Key Concepts**:
- `anomaly_score`: Higher = more unusual (0 = very normal, 1+ = highly anomalous)
- `score_threshold`: The cutoff for flagging anomalies (typically 0.5-0.7)

**Choosing the Right Threshold**:
- **Lower (0.3-0.5)**: More sensitive, catches more anomalies (higher false positives)
- **Medium (0.5-0.7)**: Balanced approach (recommended starting point)
- **Higher (0.7-0.9)**: Only flag extreme anomalies (fewer false positives)

Let's see the model's recommended threshold based on validation data:


In [ ]:
# Check the model registry for recommended threshold
registry_df = spark.table("main.default.api_anomaly_models")

print("📊 Model Metrics and Recommendations:")
registry_df.select(
    "model_name",
    "training_time",
    F.round(F.col("metrics.recommended_threshold"), 3).alias("recommended_threshold"),
    F.round(F.col("metrics.recommended_threshold_f1"), 3).alias("f1_score"),
    F.round(F.col("metrics.score_mean"), 3).alias("avg_score"),
    F.round(F.col("metrics.estimated_contamination"), 3).alias("anomaly_rate")
).show(truncate=False)

print("\n💡 Tips:")
print("   - Use 'recommended_threshold' as your starting point")
print("   - F1 score shows the balance between precision and recall")
print("   - Adjust based on your tolerance for false positives/negatives")


---
## 🔍 Part 3: Feature Importance - What Matters Most?

DQX automatically computes **feature importance** during training to show which columns contribute most to anomaly detection:


In [ ]:
# Extract feature importance from the model registry
print("📊 Feature Importance (which columns matter most):")
registry_df.select(
    "model_name",
    "feature_importance"
).show(truncate=False)

# Convert to readable format
importance = registry_df.select("feature_importance").first()["feature_importance"]
if importance:
    print("\n🎯 Feature Importance Rankings:")
    sorted_features = sorted(importance.items(), key=lambda x: x[1], reverse=True)
    for i, (feature, score) in enumerate(sorted_features, 1):
        bar = "█" * int(score * 50)
        print(f"   {i}. {feature:25} {bar} {score:.3f}")
    
    print("\n💡 Interpretation:")
    print("   - Higher scores = more important for detecting anomalies")
    print("   - Focus monitoring efforts on high-importance features")


---
## 🔄 Part 4: Drift Detection - When to Retrain?

**The Problem**: Data distributions change over time:
- API patterns evolve (new features, traffic patterns)
- Infrastructure upgrades change baselines
- Models become stale and less accurate

**The Solution**: DQX automatically detects **data drift** and warns you when retraining is needed.

Let's simulate shifted data and see drift detection in action:


In [ ]:
# Simulate data after infrastructure upgrade (everything is 20% faster now)
drifted_response_times = normal_response_times * 0.8  # 20% improvement
drifted_volumes = normal_request_volumes * 1.3  # 30% more traffic

drifted_data = [
    (float(rt), float(rv)) 
    for rt, rv in zip(drifted_response_times[:100], drifted_volumes[:100])
]

drifted_df = spark.createDataFrame(drifted_data, schema="response_time_ms double, request_volume double")

print("📊 Drifted data (after infrastructure upgrade):")
print(f"   Original mean response_time: {normal_response_times.mean():.1f}ms")
print(f"   New mean response_time:      {drifted_response_times[:100].mean():.1f}ms")
print(f"   Original mean volume:        {normal_request_volumes.mean():.1f}")
print(f"   New mean volume:             {drifted_volumes[:100].mean():.1f}")

# Apply checks on drifted data - DQX will warn about drift
result_drift = dq_engine.apply_checks(drifted_df.limit(5), checks)

print("\n✅ Check the warnings above! DQX detected drift and recommended retraining.")
print("\n💡 When you see drift warnings:")
print("   - Review if the distribution change is expected (e.g., infrastructure upgrade)")
print("   - Retrain the model on recent data: anomaly.train(df=new_data, ...)")
print("   - Or disable drift detection: drift_threshold=None")


---
## 🚀 Part 5: Advanced Features

### Ensemble Models for Better Accuracy

Train multiple models and average their predictions for more robust anomaly detection:


In [ ]:
# Train an ensemble model (5 models with different random seeds)
ensemble_params = AnomalyParams(
    ensemble_size=3,  # Train 3 models
    algorithm_config=IsolationForestConfig(
        contamination=0.1,  # Expect 10% anomalies
        num_trees=100  # Trees per model
    )
)

print("🔄 Training ensemble model (this takes a bit longer)...")
ensemble_uri = train(
    df=train_df,
    columns=["response_time_ms", "request_volume"],
    model_name="api_monitoring_ensemble",
    registry_table="main.default.api_anomaly_models",
    params=ensemble_params
)

print(f"✅ Ensemble model trained!")
print(f"\n💡 Benefits of ensemble:")
print("   - More robust to outliers")
print("   - Better generalization")
print("   - Provides confidence intervals (anomaly_score_std)")


---
## 🎓 Summary & Next Steps

### What You Learned

✅ **Basic 101**: Train and score anomalies in 3 lines of code  
✅ **Thresholds**: Interpret scores and choose the right cutoff  
✅ **Feature Importance**: Understand which columns matter most  
✅ **Drift Detection**: Automatic warnings when models need retraining  
✅ **Advanced**: Ensemble models for production use  

### Real-World Use Cases

1. **Fraud Detection**: Credit card transactions, insurance claims
2. **Infrastructure Monitoring**: Server metrics, API performance, network traffic
3. **Manufacturing**: Sensor data, equipment failures, quality control
4. **Security**: Login patterns, access logs, threat detection
5. **Business Metrics**: Sales patterns, user behavior, KPI monitoring

### Next Steps

1. **Try with Your Data**: Replace the sample data with your actual tables
2. **Integrate with Workflows**: Use `AnomalyTrainerWorkflow` for scheduled retraining
3. **Combine with Other Checks**: Use alongside traditional DQX rules
4. **Explore Temporal Features**: Add time-based patterns (hour, day_of_week, etc.)
5. **Read the Docs**: See the full guide at [DQX Documentation](https://databrickslabs.github.io/dqx/)

### 🔗 Additional Resources

- **Documentation**: `docs/dqx/docs/guide/anomaly_detection.mdx`
- **API Reference**: `docs/dqx/docs/reference/quality_checks.mdx`
- **More Examples**: `demos/anomaly_demo_standalone.py`

---

**Questions?** Open an issue on the [DQX GitHub repo](https://github.com/databrickslabs/dqx)!
